In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [2]:
import gc

In [3]:

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [4]:
#Tensorboard Things

#%load_ext tensorboard --logdir logs/fit
import tensorflow as tf
import datetime, os


In [4]:
#For the distribution of the dataset
import os
import numpy as np
import glob
import shutil
class DatasetDivision:
	def __init__(self, root_dir="",output_dir=""):
		self.root_dir = root_dir
		self.output_dir = output_dir
		print("Instance of the class created")
	def printnow(self, new_dir):
		print("Just testing that the method calling is working "+new_dir)
	def divide_dataset(self, root_dir,output_dir):
		self.root_dir =root_dir
		self.output_dir = output_dir
		if os.path.exists(self.output_dir):
			if not os.path.exists(os.path.join(self.output_dir,'train')):
				os.mkdir(os.path.join(self.output_dir,'train')) 
				os.mkdir(os.path.join(self.output_dir,'val')) 
				os.mkdir(os.path.join(self.output_dir,'test')) 
		else:
			os.mkdir(self.output_dir)
			os.mkdir(os.path.join(self.output_dir,'train')) 
			os.mkdir(os.path.join(self.output_dir, 'val')) 
			os.mkdir(os.path.join(self.output_dir, 'test'))
		# Split train/val/test sets
		for file in os.listdir(root_dir):            
			classes_path = os.path.join(root_dir, file)  
			class_files = [name for name in glob.glob(os.path.join(classes_path,'*.bmp'))]
			if class_files == []:
				continue
			train_and_valid, test = train_test_split(class_files, test_size=0.20, random_state=42)
			train, val = train_test_split(train_and_valid, test_size=0.25, random_state=42)

			#creating the training, validation and testing directories where the data will be moved.
			train_dir = os.path.join(self.output_dir, 'train',file) #creates the train data path for Divided_Dataset 
			val_dir = os.path.join(self.output_dir, 'val', file) #creates the validation data path for Divided_Dataset 
			test_dir = os.path.join(self.output_dir, 'test',file) #creates the test data path for Divided_Dataset 
			if not os.path.exists(train_dir):
				os.mkdir(train_dir)
			if not os.path.exists(val_dir):
				os.mkdir(val_dir)
			if not os.path.exists(test_dir):
				os.mkdir(test_dir)

			for frame_folders in train:
				#get only the last directory of the path frame_folders
				frame_folder = os.path.join(root_dir,file,frame_folders)
				shutil.move(frame_folder,train_dir)
			for frame_folders in val:
				frame_folder = os.path.join(root_dir,file,frame_folders)
				shutil.move(frame_folder,val_dir)
			for frame_folders in test:
				frame_folder = os.path.join(root_dir,file,frame_folders)
				shutil.move(frame_folder,test_dir)
			print('Dataset Division finished.')

In [5]:
import glob

In [6]:
smear_dir = r"D:\Pesquisa\DeepCervix\Classification of Herlev Dataset\smear2005"

In [7]:
path_dir= smear_dir
output_dir = r"D:\Pesquisa\DeepCervix\Classification of Herlev Dataset\herlev2005Format"
datasetdiv1 = DatasetDivision(path_dir,output_dir)
datasetdiv1.printnow("The new guy")
datasetdiv1.divide_dataset(path_dir, output_dir)

Instance of the class created
Just testing that the method calling is working The new guy


In [8]:
#Creating a class for data augmentation
import  cv2
import imgaug
import imageio
import os
import numpy as np
from imgaug import augmenters as iaa

class DataAugmentation:
	def __init__(self, root_dir="",output_dir=""):
		self.root_dir = root_dir
		self.output_dir = output_dir
		print("Instance of the DataAugmentation class created")
	def augmentation_of_image(self, test_image, output_path):
		self.test_image = test_image
		self.output_path = output_path
		#define the Augmenters
		#properties: A range of values signifies that one of these numbers is randmoly chosen for every augmentation for every batch

		# Apply affine transformations to each image.
		rotate = iaa.Affine(rotate=(-90,90));  
		scale = iaa.Affine(scale={"x": (0.5, 0.9), "y": (0.5,0.9)}); 
		translation = iaa.Affine(translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)})
		shear = iaa.Affine(shear=(-2, 2)); 
		zoom = iaa.PerspectiveTransform(scale=(0.01, 0.15), keep_size=True) 
		h_flip = iaa.Fliplr(1.0); 
		v_flip = iaa.Flipud(1.0); 
		padding=iaa.KeepSizeByResize(iaa.CropAndPad(percent=(0.05, 0.25)))


		#More augmentations
		blur = iaa.GaussianBlur(sigma=(0, 1.22)) 
		contrast = iaa.contrast.LinearContrast((0.75, 1.5))
		contrast_channels = iaa.LinearContrast((0.75, 1.5), per_channel=True) 
		sharpen = iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))
		gauss_noise = iaa.AdditiveGaussianNoise(scale=0.111*255, per_channel=True) 
		laplace_noise = iaa.AdditiveLaplaceNoise(scale=(0, 0.111*255)) 


		#Brightness 
		brightness = iaa.Multiply((0.35,1.65)) 
		brightness_channels = iaa.Multiply((0.5, 1.5), per_channel=0.75) 

		#CHANNELS (RGB)=(Red,Green,Blue)
		red =iaa.WithChannels(0, iaa.Add((10, 100))) 
		red_rot = iaa.WithChannels(0,iaa.Affine(rotate=(0, 45))) 
		green= iaa.WithChannels(1, iaa.Add((10, 100)))
		green_rot=iaa.WithChannels(1,iaa.Affine(rotate=(0, 45))) 
		blue=iaa.WithChannels(2, iaa.Add((10, 100)))
		blue_rot=iaa.WithChannels(2,iaa.Affine(rotate=(0, 45))) 

		#colors
		channel_shuffle =iaa.ChannelShuffle(1.0); #shuffle all images of the batch
		grayscale = iaa.Grayscale(1.0)
		hue_n_saturation = iaa.MultiplyHueAndSaturation((0.5, 1.5), per_channel=True) #change hue and saturation with this range of values for different values 
		add_hue_saturation = iaa.AddToHueAndSaturation((-50, 50), per_channel=True) #add more hue and saturation to its pixels
		#Quantize colors using k-Means clustering
		kmeans_color = iaa.KMeansColorQuantization(n_colors=(4, 16)) #quantizes to k means 4 to 16 colors (randomly chosen). Quantizes colors up to 16 colors

		#Alpha Blending 
		blend =iaa.AlphaElementwise((0, 1.0), iaa.Grayscale((0,1.0)))  #blend depending on which value is greater

		#Contrast augmentors
		clahe = iaa.CLAHE(tile_grid_size_px=((3, 21),[0,2,3,4,5,6,7])) #create a clahe contrast augmentor H=(3,21) and W=(0,7)
		histogram = iaa.HistogramEqualization() #performs histogram equalization

		#Augmentation list of metadata augmentors
		OneofRed = iaa.OneOf( [red])
		OneofGreen = iaa.OneOf( [green] )
		OneofBlue = iaa.OneOf( [blue])
		contrast_n_shit = iaa.OneOf([contrast, brightness, brightness_channels])
		SomeAug = iaa.SomeOf(2,[rotate,scale, translation, shear, h_flip,v_flip],random_order=True)
		SomeClahe = iaa.SomeOf(2, [clahe, iaa.CLAHE(clip_limit=(1, 10)),iaa.CLAHE(tile_grid_size_px=(3, 21)),iaa.GammaContrast((0.5, 2.0)),
                            iaa.AllChannelsCLAHE() , iaa.AllChannelsCLAHE(clip_limit=(1, 10), per_channel=True)],random_order=True) #Random selection from clahe augmentors
		edgedetection= iaa.OneOf([iaa.EdgeDetect(alpha=(0, 0.7)),iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0))])# Search in some images either for all edges or for directed edges.These edges are then marked in a black and white image and overlayed with the original image using an alpha of 0 to 0.7.
		canny_filter = iaa.OneOf([iaa.Canny(), iaa.Canny(alpha=(0.5, 1.0), sobel_kernel_size=[3, 7])]) #choose one of the 2 canny filter options
		OneofNoise = iaa.OneOf([blur, gauss_noise, laplace_noise])
		Color_1 = iaa.OneOf([channel_shuffle,grayscale, hue_n_saturation , add_hue_saturation, kmeans_color])
		Color_2 = iaa.OneOf([channel_shuffle,grayscale, hue_n_saturation , add_hue_saturation, kmeans_color])
		Flip = iaa.OneOf([histogram , v_flip, h_flip])

		#Define the augmentors used in the Data Augmentation
		Augmentors= [SomeAug, SomeClahe, SomeClahe, edgedetection,sharpen, canny_filter, OneofRed, OneofGreen, OneofBlue, OneofNoise, Color_1, Color_2, Flip, contrast_n_shit]


		for i in range(0,14):
			img = cv2.imread(test_image) 
			images = np.array([img for _ in range(14)], dtype=np.uint8)
			#print(images.shape)  # Prints: (14, 224, 224, 3)
			images_aug = Augmentors[i](images=images)  
			cv2.imwrite(os.path.join(output_path,test_image +"new"+str(i)+'.jpg'), images_aug[i]) 

In [9]:
class DataAugmentation_Extension:
	def __init__(self, directory=""):
		self.directory = directory
		print("Instance of DataAugmentation_Extension class created")

	def printnow(self, dir):
		print("Just testing that the method calling is working "+ dir)


	def extend_dataset(self,directory):
		#Create an instance of class 
		print("HEY")
		library_augment= DataAugmentation()
		self.directory = directory 
		if not os.path.exists(self.directory):
			print("ERROR! Couldn't find directory!")
		else:
			print("Directory exists")
		for file in os.listdir(directory):            #for any file inside the root directory 
			classes_path = os.path.join(directory, file)  #So for every folder class we create a class directory
			class_files = [name for name in glob.glob(os.path.join(classes_path,'*.BMP'))]  #alternatively we can use the globe as mentioned
			print(class_files) #call augmentation for all class_files
			for i in range(len(class_files)):
				library_augment.augmentation_of_image(class_files[i], classes_path)

In [10]:
#data augmentation
#directories
target_directory = r"D:\Pesquisa\DeepCervix\Classification of Herlev Dataset\herlev2005Format\train"
#create an instance of the class
datasetda = DataAugmentation_Extension()
datasetda.extend_dataset(target_directory)

Instance of DataAugmentation_Extension class created
HEY
Instance of the DataAugmentation class created
Directory exists
['D:\\Pesquisa\\DeepCervix\\Classification of Herlev Dataset\\herlev2005Format\\train\\carcinoma_in_situ\\149143370-149143378-001.BMP', 'D:\\Pesquisa\\DeepCervix\\Classification of Herlev Dataset\\herlev2005Format\\train\\carcinoma_in_situ\\149143370-149143378-003.BMP', 'D:\\Pesquisa\\DeepCervix\\Classification of Herlev Dataset\\herlev2005Format\\train\\carcinoma_in_situ\\149143370-149143388-002.BMP', 'D:\\Pesquisa\\DeepCervix\\Classification of Herlev Dataset\\herlev2005Format\\train\\carcinoma_in_situ\\149143370-149143388-003.BMP', 'D:\\Pesquisa\\DeepCervix\\Classification of Herlev Dataset\\herlev2005Format\\train\\carcinoma_in_situ\\149146635-149146651-001.BMP', 'D:\\Pesquisa\\DeepCervix\\Classification of Herlev Dataset\\herlev2005Format\\train\\carcinoma_in_situ\\149146635-149146651-002.BMP', 'D:\\Pesquisa\\DeepCervix\\Classification of Herlev Dataset\\herlev2

**Feature Combination**

In [6]:
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


"""LOAD DATA"""
# path to folder with data
path = r"D:\Pesquisa\DeepCervix\Classification of Herlev Dataset\herlev2005Format/"
# expected data structure inside folder: train, test, val. in each folder: one folder for each class
train_datagen = ImageDataGenerator(featurewise_center=False,
                                rotation_range = 5, fill_mode="nearest",
                                zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, # occasionally out of range
                                horizontal_flip = True, vertical_flip=True,
                                brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224), 
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

training_set_V2 =  no_DA_IDG.flow_from_directory(path+"train/", 
                                                target_size=(224, 224), 
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical', shuffle=False)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

validation_set_V2 = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""CREATE MODELS"""
from keras.models import Sequential, load_model
models_dir = r'D:\Pesquisa\DeepCervix\Classification of Herlev Dataset\Saved Model Links\smearseven\7classmodel'
model_vgg = load_model(os.path.join(models_dir,'model_vgg16_herlev.h5')) 
model_xcep = load_model(os.path.join(models_dir,'model_xcep_herlev.h5'))
model_vgg19 = load_model(os.path.join(models_dir,'model_vgg19_herlev.h5'))
model_res50 = load_model(os.path.join(models_dir,'R50_Herlev_7class.h5'))

"""LOADING WEIGHTS FROM PREVIOUSLY TRAINED MODELS"""
model_xcep.load_weights(os.path.join(models_dir,"K_xcep_T2.h5"))
model_vgg.load_weights(os.path.join(models_dir,"K_VGG16_herlev_T2.h5"))
model_res50.load_weights(os.path.join(models_dir,"K_R50_herlev.h5"))
model_vgg19.load_weights(os.path.join(models_dir,"K_VGG19_T2.h5"))

Found 8190 images belonging to 7 classes.
Found 8190 images belonging to 7 classes.
Found 185 images belonging to 7 classes.
Found 185 images belonging to 7 classes.
Found 186 images belonging to 7 classes.


In [27]:
training_set.classes

array([0, 0, 0, ..., 6, 6, 6])

In [7]:
"""MODEL COMBINATION"""
# creating the model extracting features from the last layer before the softmax layer.
vgg_extractor = Model(inputs=model_vgg.input, outputs=model_vgg.get_layer("dense_1024").output)
xcep_extractor = Model(inputs=model_xcep.input, outputs=model_xcep.get_layer("dense_1024").output)
vgg19_extractor = Model(inputs=model_vgg19.input, outputs=model_vgg19.get_layer("dense_1024").output)
r50_extractor = Model(inputs=model_res50.input, outputs=model_res50.get_layer("dense_1024").output)


In [8]:

# using the model extractor to generate feature arrays with 4096 features from each model extractor.
y_train = to_categorical(training_set_V2.classes)
X_train_m1 = vgg_extractor.predict(training_set_V2, steps = training_set_V2.n)
X_train_m2 = xcep_extractor.predict(training_set_V2, steps = training_set_V2.n)
X_train_m3 = vgg19_extractor.predict(training_set_V2, steps = training_set_V2.n)
X_train_m4 = r50_extractor.predict(training_set_V2, steps = training_set_V2.n)
X_train = np.concatenate([X_train_m1, X_train_m2, X_train_m3, X_train_m4], axis=1)

y_val = to_categorical(validation_set_V2.classes)
X_val_m1 = vgg_extractor.predict(validation_set_V2, steps = validation_set_V2.n)
X_val_m2 = xcep_extractor.predict(validation_set_V2, steps = validation_set_V2.n)
X_val_m3 = vgg19_extractor.predict(validation_set_V2, steps = validation_set_V2.n)
X_val_m4 = r50_extractor.predict(validation_set_V2, steps = validation_set_V2.n)
X_val = np.concatenate([X_val_m1, X_val_m2, X_val_m3, X_val_m4], axis=1)

y_test = to_categorical(test_set_V2.classes)
X_test_m1 = vgg_extractor.predict(test_set_V2, steps = test_set_V2.n)
X_test_m2 = xcep_extractor.predict(test_set_V2, steps = test_set_V2.n)
X_test_m3 = vgg19_extractor.predict(test_set_V2, steps = test_set_V2.n)
X_test_m4 = r50_extractor.predict(test_set_V2, steps = test_set_V2.n)
X_test = np.concatenate([X_test_m1, X_test_m2, X_test_m3, X_test_m4], axis=1)


186/186 [==============================] - 2s 8ms/step


In [9]:
from keras.callbacks import EarlyStopping

In [11]:


"""TRAIN & TEST FEATURE EXTRACTION MODEL"""
# the feature arrays are read into a sequential model directly connecting them to the softmax layer, with
# some dropout and batch normalization in between.
np.random.seed(668)

opt = Adam(learning_rate=1e-3)
model = Sequential()
model.add(Dropout(0.5, input_shape=(4096,)))
model.add(BatchNormalization())
model.add(Dense(7, activation="softmax"))
model1=model
model1.compile(optimizer=opt, loss='categorical_crossentropy', 
            metrics=['acc'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop = EarlyStopping(monitor='loss', 
                patience=20, verbose=1,
                mode='min',
                restore_best_weights=True)
callbacks = [
    tensorboard_callback,
    # stop training when val_loss does not improve for 20 epochs
]
epochs = 200
history1=model1.fit(X_train, y_train, batch_size=32, epochs=epochs,
                    verbose=2, shuffle=True, validation_data=(X_val, y_val),callbacks=stop)


Epoch 1/200
256/256 - 1s - loss: 0.5871 - acc: 0.7974 - val_loss: 0.4608 - val_acc: 0.8541 - 1s/epoch - 5ms/step
Epoch 2/200
256/256 - 1s - loss: 0.4187 - acc: 0.8482 - val_loss: 0.5044 - val_acc: 0.8432 - 715ms/epoch - 3ms/step
Epoch 3/200
256/256 - 1s - loss: 0.3590 - acc: 0.8713 - val_loss: 0.5247 - val_acc: 0.8757 - 780ms/epoch - 3ms/step
Epoch 4/200
256/256 - 1s - loss: 0.3409 - acc: 0.8790 - val_loss: 0.4896 - val_acc: 0.8324 - 758ms/epoch - 3ms/step
Epoch 5/200
256/256 - 1s - loss: 0.3203 - acc: 0.8879 - val_loss: 0.5401 - val_acc: 0.8378 - 719ms/epoch - 3ms/step
Epoch 6/200
256/256 - 1s - loss: 0.2972 - acc: 0.8945 - val_loss: 0.5299 - val_acc: 0.8270 - 712ms/epoch - 3ms/step
Epoch 7/200
256/256 - 1s - loss: 0.2876 - acc: 0.8960 - val_loss: 0.5424 - val_acc: 0.8162 - 717ms/epoch - 3ms/step
Epoch 8/200
256/256 - 1s - loss: 0.2837 - acc: 0.8999 - val_loss: 0.5538 - val_acc: 0.8486 - 718ms/epoch - 3ms/step
Epoch 9/200
256/256 - 1s - loss: 0.2779 - acc: 0.9042 - val_loss: 0.5564 - 

In [63]:
#del model,model1
#K.clear_session()
#gc.collect(2)

2435

In [20]:

"""EVALUATE MODEL ON TEST DATA"""
y_test = test_set_V2.classes
pred = np.argmax(model1.predict(X_test), axis=1)
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred,digits=4))

6/6 [==============================] - 0s 1ms/step
0.8655913978494624
[[24  0  0  1  0  0  5]
 [ 0 33  4  0  0  0  0]
 [ 0  5 24  0  0  0  1]
 [ 1  0  0 17  0  0  2]
 [ 0  0  0  0 13  1  0]
 [ 0  0  0  0  1 14  0]
 [ 2  1  1  0  0  0 36]]
              precision    recall  f1-score   support

           0     0.8889    0.8000    0.8421        30
           1     0.8462    0.8919    0.8684        37
           2     0.8276    0.8000    0.8136        30
           3     0.9444    0.8500    0.8947        20
           4     0.9286    0.9286    0.9286        14
           5     0.9333    0.9333    0.9333        15
           6     0.8182    0.9000    0.8571        40

    accuracy                         0.8656       186
   macro avg     0.8839    0.8720    0.8768       186
weighted avg     0.8678    0.8656    0.8655       186



In [21]:
model1.save(os.path.join(models_dir,
                        'model_vgg16_vgg19_Res50_xception_herlev7class.keras'))